In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
import os
import joblib
import json
import pywt
import shutil

In [5]:
face_cascade = cv2.CascadeClassifier('./haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./haarcascades/haarcascade_eye.xml')
model = joblib.load('./saved_model_sgd.pkl')
original_images = './images_dataset/'
cropped_images = './images_cropped/'
with open("./celebrity_key_dict.json") as f :
    celebrities_dic = json.load(f)

In [4]:
def get_cropped_image_if_1_face(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,1.3,5)
    if len(faces) >= 1:
        for (x,y,w,h) in faces:
            roi_color = img[y:y+h,x:x+w]
            return roi_color
    else:
        print(f'{len(faces)} face are present in image : {path}')
    return None

def get_cropped_image_if_1_face_and_2_eyes(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,1.3,5)
    if len(faces) == 1:
        for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h,x:x+w]
            roi_color = img[y:y+h,x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            if len(eyes) >= 2:
                return roi_color
            else:
                print(f'{len(eyes)} eye are present in image : {path}')
    else:
        print(f'{len(faces)} face are present in image : {path}')
    return None

def wavelet_generaotor(img, mode="haar",level=1):
    inArray = img
    inArray = cv2.cvtColor(inArray,cv2.COLOR_RGB2GRAY)
    inArray = np.float32(inArray)
    # Compute And Process Coefficients
    coeffs = pywt.wavedec2(inArray,mode,level=level)
    coeffs_H = list(coeffs)
    coeffs_H[0] *= 0
    # Reconstuction
    inArray_H = pywt.waverec2(coeffs_H,mode)
    inArray_H *= 255
    inArray_H = np.uint8(inArray_H)
    return inArray_H

def predict(img_i):
    img  = get_cropped_image_if_1_face(img_i)
    if img is not None:
        scalled_raw_img = cv2.resize(img,(32,32))
        img_wavlet_form=wavelet_generaotor(img,"db1",5)
        scalled_img_wavlet_form = cv2.resize(img_wavlet_form,(32,32))
        stacked_image = np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_wavlet_form.reshape(32*32,1)))
        stacked_image = stacked_image.reshape(1,stacked_image.shape[0])
        key = model.predict(stacked_image)[0]
        for i in celebrities_dic:
            if celebrities_dic[i] == key :
                print(i)
                break
    else:
        print("Error")

In [9]:
# Retrain
name_of_retrain_model = "elon_musk"
currennt_dir_for_cropped_images = cropped_images+name_of_retrain_model+'/'


if os.path.exists(currennt_dir_for_cropped_images):
    shutil.rmtree(currennt_dir_for_cropped_images)
os.mkdir(currennt_dir_for_cropped_images)
for imgs in os.scandir(original_images+name_of_retrain_model+'/'):
    if imgs.is_file():
        tmp = get_cropped_image_if_1_face_and_2_eyes(imgs.path)
        if tmp is None:
            pass
        else :
            try :
                cv2.imwrite(currennt_dir_for_cropped_images+imgs.name,tmp)         
            except :
                print(f"Save Failed : {currennt_dir_for_cropped_images+imgs.name}")
                print("Retrying.....")
                try:   
                    filename = currennt_dir_for_cropped_images+imgs.name
                    filename = os.path.splitext(filename)[0]+".jpg"
                    cv2.imwrite(filename,tmp)         
                except:
                    print(f"Save Failed : {currennt_dir_for_cropped_images+imgs.name}")

1 eye are present in image : ./images_dataset/elon_musk/7576ecd516344ef1b9698d459047f1f7.jpg
2 face are present in image : ./images_dataset/elon_musk/elon-musk (1).webp
2 face are present in image : ./images_dataset/elon_musk/Elon-Musk-2010.jpg
0 eye are present in image : ./images_dataset/elon_musk/GettyImages-1181925987.jpg
0 face are present in image : ./images_dataset/elon_musk/images.jpg
2 face are present in image : ./images_dataset/elon_musk/los-angeles-feb-26-elon-260nw-591307859.webp
1 eye are present in image : ./images_dataset/elon_musk/musk-gty-rc-200909_hpMain_16x9t_608.jpg


In [11]:
celebrities_dic['elon_musk']=5

In [15]:
with open("./celebrity_key_dict.json","w") as f :
    f.write(json.dumps(celebrities_dic))

In [21]:
# List All Files
trainingfilelist = []
for j in os.scandir(currennt_dir_for_cropped_images):
    trainingfilelist.append(j.path)
trainingfilelist

['./images_cropped/elon_musk/105773423-1551716977818rtx6p9yw.jpg',
 './images_cropped/elon_musk/1206292068.jpg.jpg',
 './images_cropped/elon_musk/1340px-elon_musk_presenting_teslas_fully_autonomous_future_40705940233_10_0.jpg',
 './images_cropped/elon_musk/180515-10thingselonmusk-editorial.jpg',
 './images_cropped/elon_musk/200131-elon-musk-al-1440_85b17128253456024fdcc368353c8d0e.jpg',
 './images_cropped/elon_musk/960x0.jpg',
 './images_cropped/elon_musk/ac8b25d14dbf6b808c0c84527338b04d.jpg',
 './images_cropped/elon_musk/AP_20322727772083.jpg',
 './images_cropped/elon_musk/elon-musk.webp',
 './images_cropped/elon_musk/ElonMusk-RTX2J8M (1).jpg',
 './images_cropped/elon_musk/elon_musk_royal_society (1).jpg',
 './images_cropped/elon_musk/Elon_Musk_Royal_Society.jpg',
 './images_cropped/elon_musk/gettyimages-930522554-612x612.jpg',
 './images_cropped/elon_musk/getty_489769706_200013332000928069_437741.jpg',
 './images_cropped/elon_musk/MUS00005-Elon-Musk (1).jpg',
 './images_cropped/elon_

In [22]:
X = []
Y = []
for training_image in trainingfilelist:
    img  = cv2.imread(training_image)
    scalled_raw_img = cv2.resize(img,(32,32))
    img_wavlet_form=wavelet_generaotor(img,"db1",5)
    scalled_img_wavlet_form = cv2.resize(img_wavlet_form,(32,32))
    stacked_image = np.vstack((scalled_raw_img.reshape(32*32*3,1),scalled_img_wavlet_form.reshape(32*32,1)))
    X.append(stacked_image)
    Y.append(celebrities_dic['elon_musk'])

In [24]:
Y

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [29]:
np.array(X).shape

(19, 4096, 1)

In [30]:
X[0].shape

(4096, 1)

In [31]:
X = np.array(X).reshape(len(X),X[0].shape[0]).astype(float)

In [33]:
X.shape

(19, 4096)

In [36]:
model.partial_fit(X,Y,classes=np.unique(Y))

ValueError: `classes=array([5])` is not the same as on last call to partial_fit, was: array([0, 1, 2, 3, 4])

In [35]:
Y

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]